In [22]:
### 필요한 함수 임폴트
import os
from dotenv import load_dotenv
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core import Settings, Document, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
import tqdm
import pandas as pd
import json
import pickle
from dotenv import load_dotenv
import google.generativeai as generativeai
import random
import time
import re

### 리뷰 통합
file_path = '데이터분석/input/통합데이터(간단한_텍스트_전처리)_sample.csv'

df = pd.read_csv(file_path)

# groupby를 이용하여 리뷰 묶기
df_grouped = (
    df.groupby('고유번호')
      .agg({
          '제품명': 'first',
          '상세정보': 'first',
          '카테고리': 'first',
          '리뷰': lambda x: ' / '.join(x.fillna('').astype(str))
      })
      .reset_index()
)

df_grouped.to_csv('outputs/1.gemini_리뷰통합_result.csv',index=False)


### df_grouped 데이터 프레임 임폴트

file_path = '데이터분석/input/리뷰통합_sample.csv'

df_grouped = pd.read_csv(file_path)

### .env file 로드
load_dotenv('.env')

### gemini key 불러오기기
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
generativeai.configure(api_key=GOOGLE_API_KEY)

# 리뷰 텍스트 생성
review_text = df_grouped.loc[:, '리뷰'].to_list()

### 감성 분석 및 근거가 되는 keyword 추출 함수 정의  --> 12시간 걸림
def analyze_review(text_input):
    # prompt 생성
    prompt = f"""
    다음 텍스트는 건강기능식품에 관한 소비자 리뷰입니다. 제품 하나당 모든 리뷰가 '/'로 구분되어져 포함되어 있습니다.
    해당 리뷰의 내용에서 긍정인 내용, 부정인 내용, 그리고 중립인 내용을 분류를 하고, 각 감성별로 리뷰를 요약하여 JSON 형식으로 제시해 주세요.
    각 감성별 리뷰 요약은 맥락이 비슷한 것만 남기고 요약을 하여, 각 감성별 리뷰의 글자 수가 300자를 넘지 않도록 요약해 주세요.
    그리고 리뷰 요약할 때 없는 내용을 창조하면 안되고, 리뷰 내용에 100% 충실하게 요약해야 해야 합니다.
    JSON 형식만 출력하세요. 아래 형식 외의 설명 문구는 절대 쓰지 마세요.

    텍스트: {text_input}

    출력 형식 예시:
    {{
      "정확도": 85,
      "감성별 리뷰 요약": {{
        "긍정": ["긍정 리뷰 요약"],
        "부정": ["부정 리뷰 요약"],
        "중립": ["중립 리뷰 요약"]
      }}
    }}
    """


    try:
        # 텍스트 분석 결과 생성
        model = generativeai.GenerativeModel("gemini-2.5-pro")

        # generation_config 객체를 생성하여 temperature를 설정합니다.
        # temperature 값은 0.0 (가장 보수적)에서 1.0 (가장 창의적) 사이로 조절할 수 있습니다.
        generation_config = generativeai.types.GenerationConfig(
            temperature=0.0,  # 원하는 temperature 값으로 변경하세요 (예: 0.2, 0.5, 0.9 등)
            top_p=0.9,
            top_k=50
        )

        response = model.generate_content(
            contents=[prompt],
            generation_config=generation_config # 여기에 generation_config를 전달합니다.
        )


        result_str = response.text
        if result_str:
            # print(f'결과 : {result_str}')

            # JSON 부분만 추출
            match = re.search(r"\{[\s\S]*\}", result_str)
            if match:
                json_text = match.group(0).strip()
                try:
                    parsed_json = json.loads(json_text)
                    return parsed_json
                except json.JSONDecodeError:
                    print("JSON 디코딩 실패. 추출된 내용:", json_text)
                    return None
            else:
                print("JSON 패턴을 찾지 못했습니다.")
                return None


    except Exception as e:
        print(f'API 호출 오류 : {e}')
        return None



### 전체 텍스트를 처리하는 함수 정의 (tqdm 적용)
def process_multiple_texts(text_list):
    results = {}
    for i, text_input in enumerate(tqdm(text_list, desc="리뷰 처리 진행률")):
        retry_count = 0
        max_retries = 5  # 최대 재시도 횟수
        wait_time = 20  # 초기 대기 시간 (초)

        while retry_count < max_retries:
            print(f"텍스트 {i+1} 처리 시도 {retry_count + 1}...")
            result = analyze_review(text_input)
            if result:
                # 성공 조건을 "올바른 딕셔너리(JSON 파싱 성공)"로 한정
                if isinstance(result, dict):
                    results[f"텍스트 {i+1}"] = result
                    break  # 성공 시 루프 종료
                else:
                    retry_count += 1
                    wait_time = wait_time * 2 + random.uniform(0, 1)
                    print(f"API 요청 또는 JSON 파싱 실패. {wait_time:.2f}초 후 재시도...")
                    time.sleep(wait_time)

        else:  # 최대 재시도 횟수 초과 시
            results[f"텍스트 {i+1}"] = "감정 키워드 추출 실패 (최대 재시도 횟수 초과)"

    return results


# 여러 텍스트 처리
all_results = process_multiple_texts(review_text)

# 최종 결과를 dict 자료 구조로 변환
with open('review_analysis.pkl', 'wb') as fw:
    pickle.dump(all_results, fw)

# 저장된 결과를 다시 불러오기
with open('review_analysis.pkl', 'rb') as fr:
    loaded_data = pickle.load(fr)

# print(f'리뷰 데이터 분석의 결과 : \n{loaded_data}')

### DataFrame에 붙이기

df_grouped["리뷰_분석"] = list(loaded_data.values())


### 감성별로 리뷰 따로 떼서 열 만들기 (임베딩 용)

# 긍정/부정/중립 리뷰 요약만 새 열로 추가
df_grouped['긍정_리뷰'] = df_grouped['리뷰_분석'].apply(lambda x: x['감성별 리뷰 요약']['긍정'] if isinstance(x, dict) else None)
df_grouped['부정_리뷰'] = df_grouped['리뷰_분석'].apply(lambda x: x['감성별 리뷰 요약']['부정'] if isinstance(x, dict) else None)
df_grouped['중립_리뷰'] = df_grouped['리뷰_분석'].apply(lambda x: x['감성별 리뷰 요약']['중립'] if isinstance(x, dict) else None)

# df_grouped에서 '리뷰' 열 삭제

df_grouped.drop(columns='리뷰', inplace=True)

# csv 저장

df_grouped.to_csv('outputs/2.gemini_리뷰분석_result.csv', index=False)

# csv 확인

import pandas as pd

file_path = '데이터분석/output/6.리뷰분석(감성_리뷰만 있는 버전)_sample.csv'

df = pd.read_csv(file_path)

print(df)

,고유번호,제품명,카테고리,상세정보,긍정_리뷰,부정_리뷰,중립_리뷰
0,10067386235,Newt365,유산균,NaN,섭취 후 변비가 해소되어 매일 화장실을 가게 되었다는 등 배변 활동에 대한 긍정적 ...,"광고와 달리 효과를 전혀 보지 못했거나, 처음에는 효과가 있다가 사라졌다는 의견이 ...","제품을 막 받아서 아직 복용 전이거나, 복용한 지 얼마 되지 않아 효과를 기대하며 ..."
1,10102052354,Sentroom_man,비타민,"주요 기능성(식약처인증) 영양보충 영양소 원료명(식약처고시) 비타민A, 비타민D, ...","많은 소비자들이 피로 회복과 컨디션 개선 효과를 경험했으며, 먹은 날과 안 먹은 날...",일부 소비자들은 알약의 크기가 커서 목 넘김이 불편하다고 지적했습니다. 복부팽만감과...,제품 효과에 대해 아직 잘 모르겠다는 의견이 다수 있었습니다. 꾸준히 복용하고 있지...
2,10102054620,Sentroom_woman,비타민,"주요 기능성(식약처인증) 영양보충 영양소 원료명(식약처고시) 비타민A, 비타민D, ...","꾸준히 재구매하는 소비자가 많으며, 피로 개선과 활력 증진에 효과를 느꼈다는 평이 ...","일부 소비자들은 알약의 크기가 커서 목 넘김이 불편하다고 지적했습니다. 또한, 배송...",제품을 처음 구매하여 효과를 기대하며 꾸준히 복용해보겠다는 다짐을 남긴 리뷰가 많습...
3,10158604245,Theday,아연,"주요 기능성(식약처인증) 면역력 영양소 원료명(식약처고시) 아연, 비타민D, 비타민...","아이들이 망고맛 젤리 형태라 맛있게 잘 먹고 좋아하며, 스스로 찾아 먹거나 더 달라...","일부 아기들은 젤리 형태나 식감에 익숙하지 않아 먹는 것을 어색해하거나, 씹기 어려...","제품을 구매한 지 얼마 되지 않아 아직 효과는 잘 모르겠지만, 아이의 면역력이 좋아..."
4,10164573896,Tenten,비타민,"주요 기능성(식약처인증) 영양보충 영양소 원료명(식약처고시) 비타민B6, 비타민B2...",아이들이 딸기맛 젤리 같아 거부감 없이 아주 잘 먹는다는 평이 지배적입니다. 다른 ...,약국에서 판매하는 제품과 성분 및 함량이 다른 '텐텐맛' 제품이라 속았다는 불만이 ...,배송이 빠르고 상품을 잘 받았다는 단순 수령 확인 및 감사 인사가 주를 이룹니다. ...
5,10197554401,Sentroom_multi,비타민,"주요 기능성(식약처인증) 영양보충 영양소 원료명(식약처고시) 비타민A, 비타민E, ...",아이들이 젤리처럼 맛있게 잘 먹어 스스로 챙겨 먹는다는 리뷰가 압도적으로 많습니다....,"배송 중 박스가 파손되거나, 더운 날씨로 인해 젤리가 서로 달라붙거나 녹아서 도착했...","제품의 표지나 포장이 변경되었다는 내용과, 유통기한이 넉넉하다는 사실을 언급한 리뷰..."
6,102512148,Vitaheim,비타민,"주요 기능성(식약처인증) 피로회복 영양소 원료명(식약처고시) 칼슘, 마그네슘, 비타...","빠른 배송, 저렴한 가격, 꼼꼼한 포장에 대한 만족도가 높습니다. 맛이 좋고(레몬,...",제품이 물에 잘 녹지 않고 가루나 침전물이 남아 먹기 불편하다는 의견이 있습니다. ...,"'괜찮다' 또는 '그냥 그렇다' 정도의 중립적인 반응과 '잘 받았다', '잘 먹겠다..."
7,10257670571,Eatthefit,단백질,NaN,"초코우유나 초코에몽 같은 진하고 맛있는 '속세의 맛'이라는 평이 지배적이며, 후레이...","가격이 비싸다는 의견이 가장 많았으며, 양이 적어 아쉽다는 지적도 있었습니다. 일부...",배송이 빠르다는 점과 스푼이 동봉되어 편리하다는 내용이 반복적으로 언급되었습니다. ...
8,10258433169,Nutricore,오메가3,주요 기능성(식약처인증) 혈행개선 영양소 원료명(식약처고시) 해당 없음 DHA+EP...,"제품 효과에 대한 만족도가 높습니다. 손발 저림, 안구 건조증, 피로감 개선 등 구...",가격이 비싸다는 의견이 가장 많습니다. 다른 판매처와의 가격 차이나 가성비에 대한 ...,제품을 처음 구매하여 꾸준히 먹어보겠다는 다짐과 함께 효과에 대한 기대감을 나타내는...
9,10258863076,Extreme,비타민,"주요 기능성(식약처인증) 영양보충 영양소 원료명(식약처고시) 비타민A, 엽산, 비타...",한 포에 여러 영양제가 들어있어 간편하게 섭취할 수 있다는 점이 가장 큰 장점으로 ...,"일부 알약의 크기가 너무 커서 목 넘김이 불편하다는 의견이 있습니다. 또한, 개별 ...",제품을 이제 막 먹기 시작했거나 선물용으로 구매하여 아직 효과를 잘 모르겠다는 리뷰...
